In [13]:
#import os
import numpy as np
#from google.colab import drive
from sklearn.preprocessing import scale
#drive.mount('/content/drive')

In [14]:
#os.chdir("/content/drive/MyDrive/Colab Notebooks/ass4")

In [21]:
train_data = np.loadtxt('parkinsonsTrainStatML.dt')
test_data = np.loadtxt('parkinsonsTestStatML.dt')

def splitlabel(data):
    label = data[:,data.shape[1]-1]
    features = np.delete(data, data.shape[1]-1, axis=1)
    return(label, features)

def normalizedata(trainset,testset):
    (trainlabel,traindata) = splitlabel(trainset)
    (testlabel,testdata) = splitlabel(testset)
    mean = np.mean(traindata,axis=0)
    std = np.std(traindata,axis=0)
    norm_train = (traindata - mean) / std
    norm_test = (testdata - mean) / std
    np.savetxt(
      "mean and var.csv", np.vstack((
        np.mean(traindata, axis=0), 
        np.var(traindata, axis=0), 
        np.around(np.mean(norm_test, axis=0),2), 
        np.var(norm_test, axis=0)
        )), delimiter=','
        )
    return norm_train, trainlabel, norm_test, testlabel

x, y, x_test, y_test = normalizedata(train_data,test_data)

In [16]:
# set C and gamma
C = [10**n for n in range(-2,6)]
gamma = [10**n for n in range(-2,6)]

# cross-val
from sklearn import svm
from sklearn.model_selection import cross_val_score, KFold

cv_scores = []
hp = []
k_fold = KFold(n_splits=5, shuffle=True, random_state=1)
for i in C:
  for j in gamma:
    clf = svm.SVC(kernel='rbf', C=i, gamma=j)
    pra = [i,j]
    scores = cross_val_score(clf, x, y, cv=5, scoring = 'accuracy')
    cv_scores.append(scores.mean())
    hp.append(pra)

# find best hyper-parameter
besthp = hp[np.argmax(cv_scores)] 
print(besthp,max(cv_scores))

[10, 0.1] 0.908421052631579


In [17]:
# use GridSearchCV
from sklearn.model_selection import GridSearchCV

param=[{"kernel":["rbf"],"C":[10**n for n in range(-2,6)], "gamma": [10**n for n in range(-2,6)]}]
grid = GridSearchCV(svm.SVC(), param_grid=param, cv=5)
grid.fit(x,y)
print('best_params:',  grid.best_params_)  
print('best_score_:', grid.best_score_) 

best_params: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
best_score_: 0.908421052631579


In [18]:
# train model
model = svm.SVC(kernel='rbf', C=10, gamma=0.1)
model.fit(x,y)
# predict
y_pred = model.predict(x_test)

# Compute training and test error
print('training error',1 - model.fit(x, y).score(x, y))
from sklearn.metrics import accuracy_score
print('test error',1- accuracy_score(y_test, y_pred))

training error 0.0
test error 0.09278350515463918


In [19]:
# C and SV
nums_bsv = []
nums_free = []
for i in [10**n for n in range(-2,6)]:
  model_c = svm.SVC(kernel='rbf', C=i, gamma=0.1)
  model_c.fit(x,y)
  y_pred_c = model_c.predict(x_test)
  alphas = np.abs(model_c.dual_coef_)
  #print(alphas)
  free = [i , alphas[np.where(alphas < i)].shape[0]]
  bsv = [i , alphas[np.where(alphas == i)].shape[0]]
  nums_bsv.append(bsv)
  nums_free.append(free)

In [20]:
print('number of bounded SV:', nums_bsv)
print('number of free SV:', nums_free)

number of bounded SV: [[0.01, 39], [0.1, 38], [1, 25], [10, 1], [100, 0], [1000, 0], [10000, 0], [100000, 0]]
number of free SV: [[0.01, 29], [0.1, 34], [1, 40], [10, 55], [100, 54], [1000, 54], [10000, 54], [100000, 54]]
